In [3]:
import pandas as p

query = 'select \
M.id as match_id \
,M.date as match_date \
,L.name as league_name \
,M.stage as match_stage \
,C.name as country_name \
,M.season as Season \
,HT.team_long_name as home_team_long_name \
,HT.team_short_name as home_team_short_name \
,AT.team_long_name as away_team_long_name \
,AT.team_short_name as away_team_short_name \
,M.home_team_goal as home_team_goals \
,M.away_team_goal as away_team_goals \
,M.home_player_1 \
,M.home_player_2 \
,M.home_player_3 \
,M.home_player_4 \
,M.home_player_5 \
,M.home_player_6 \
,M.home_player_7 \
,M.home_player_8 \
,M.home_player_9 \
,M.home_player_10 \
,M.home_player_11 \
,M.away_player_1 \
,M.away_player_2 \
,M.away_player_3 \
,M.away_player_4 \
,M.away_player_5 \
,M.away_player_6 \
,M.away_player_7 \
,M.away_player_8 \
,M.away_player_9 \
,M.away_player_10 \
,M.away_player_11 \
,M.goal \
,M.shoton \
,M.shotoff \
,M.corner \
,M.foulcommit \
,M.card \
,M.cross \
,M.possession \
from Match as M \
join League as L on M.league_id = L.id \
join Country as C on C.id = L.country_id \
join Team as HT on HT.team_api_id = M.home_team_api_id \
join Team as AT on AT.team_api_id = M.away_team_api_id'

print(query)

select M.id as match_id ,M.date as match_date ,L.name as league_name ,M.stage as match_stage ,C.name as country_name ,M.season as Season ,HT.team_long_name as home_team_long_name ,HT.team_short_name as home_team_short_name ,AT.team_long_name as away_team_long_name ,AT.team_short_name as away_team_short_name ,M.home_team_goal as home_team_goals ,M.away_team_goal as away_team_goals ,M.home_player_1 ,M.home_player_2 ,M.home_player_3 ,M.home_player_4 ,M.home_player_5 ,M.home_player_6 ,M.home_player_7 ,M.home_player_8 ,M.home_player_9 ,M.home_player_10 ,M.home_player_11 ,M.away_player_1 ,M.away_player_2 ,M.away_player_3 ,M.away_player_4 ,M.away_player_5 ,M.away_player_6 ,M.away_player_7 ,M.away_player_8 ,M.away_player_9 ,M.away_player_10 ,M.away_player_11 ,M.goal ,M.shoton ,M.shotoff ,M.corner ,M.foulcommit ,M.card ,M.cross ,M.possession from Match as M join League as L on M.league_id = L.id join Country as C on C.id = L.country_id join Team as HT on HT.team_api_id = M.home_team_api_id join

In [5]:
import pandas as pd
import sqlite3 as db
conn = db.connect("database.sqlite")

#Create match dataframe with match query
df_match = pd.read_sql_query(query, conn)

In [10]:
#df_match.info()
len(df_match.columns)

42

In [ ]:
#df[df['home_team_goals'] == df['away_team_goals']]['match_stage'].value_counts()
df['match_stage'].value_counts()

In [ ]:
def get_winning_team(df):
    if df['home_team_goals'] > df['away_team_goals'] :
        return df['home_team_short_name']
    elif df['home_team_goals'] < df['away_team_goals']:
        return df['away_team_short_name']
    else:
        return 'No result'

df['winning_team'] = df.apply(get_winning_team, axis=1)

df_only_win_matches = df.query('winning_team != "No result"')
#df_only_win_matches['winning_team'].value_counts()


    

In [ ]:
df['match_date'] = pd.to_datetime(df['match_date'], infer_datetime_format=True)

In [ ]:
df.columns

In [ ]:
# Q1: Which team improved the most?

#We need to find the wins vs total matches for the 1st season, and the last season and compare the results

#First we need to compute the list of matches

import numpy as np
df.columns

#For computing the list of matches for all the team names

#Get home games
home_games = df.groupby(['Season', 'home_team_short_name']).count()['match_id'].reset_index()
home_games.rename(index=str, columns={"match_id": "home_match_count"}, inplace=True)


#Get away games
away_games = df.groupby(['Season', 'away_team_short_name']).count()['match_id'].reset_index()
away_games.rename(index=str, columns={"match_id": "away_match_count"}, inplace=True)
away_games

all_matches = pd.merge(home_games, away_games,  how='inner', left_on=['Season','home_team_short_name'], right_on = ['Season','away_team_short_name'])
all_matches.rename(index=str, columns={"home_team_short_name": "team_short_name"}, inplace=True)
all_matches.drop('away_team_short_name',axis=1, inplace=True)
all_matches

#testing for a team to make sure it works
sum((df['Season'] == '2015/2016') & ((df['home_team_short_name'] == 'ZWO') | ( df['away_team_short_name'] == 'ZWO')))

all_matches['total_matches'] = all_matches['home_match_count'] + all_matches['away_match_count']
all_matches

#compute winning teams 
interested_column_index = np.r_[np.arange(0,3), 4, 5, 42]
interested_column_index
df_interested_columns = df.iloc[:,interested_column_index]
winning_matches.rename(index=str, columns={'winning_team': 'team_short_name'}, inplace=True)

#Remove the rows with no result
df_interested_columns = df_interested_columns.query('winning_team != "No result"')
winning_matches = df_interested_columns.groupby(['Season', 'winning_team']).count()['match_id'].reset_index()
winning_matches.rename(index=str, columns={'match_id': 'win_match_count'}, inplace=True)
winning_matches

all_data = pd.merge(all_matches, winning_matches, how='inner', left_on=['Season','team_short_name'], right_on=['Season', 'winning_team'])
all_data.drop('winning_team',axis=1, inplace=True)
all_data['win_percentage' ] =  all_data['win_match_count']/all_data['total_matches']
all_data
pivoted_data = all_data.pivot(index='team_short_name', columns='Season', values='win_percentage').reset_index()
pivoted_data.dropna(inplace=True)
pivoted_data.set_index('team_short_name', inplace=True)

pivoted_data.columns = pivoted_data.columns.str.strip().str.lower().str.replace('/', '_')
pivoted_data['overall_win_percentage_change'] = pivoted_data['2015_2016'] - pivoted_data['2008_2009']
pivoted_data = pivoted_data.sort_values(by='overall_win_percentage_change', ascending=False)
pivoted_data.head(1)
#Compute the difference between 1st season ()

In [ ]:

#| df['away_team_short_name'] == 'ZWO')